<a href="https://colab.research.google.com/github/AkashBR/Audio-Classification/blob/main/Cluster930.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the CSV file"
df = pd.read_csv('/content/drive/My Drive/nifty_calculated_data.csv')

Mounted at /content/drive


In [18]:
df['time'] = pd.to_datetime(df['time'], format='%H:%M').dt.time

In [24]:
import numpy as np

def assign_cluster(value):
    if pd.isna(value):
        return np.nan
    for i in np.arange(-10.0, 10.1, 0.1):
        if i <= value < i + 0.1:
            return f"{i:.1f}-{i+0.1:.1f}"
    return np.nan

In [27]:
# Convert time to datetime.time format
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

# Filter for 9:30 AM data
df_930 = df[df['time'] == pd.to_datetime('09:30:00').time()]

In [28]:
def assign_cluster(value):
    if pd.isna(value):
        return np.nan
    for i in np.arange(-10.0, 10.1, 0.1):
        if i <= value < i + 0.1:
            return f"{i:.1f}-{i+0.1:.1f}"
    return np.nan

# Create the cluster930 column for 9:30 AM data
df.loc[df['time'] == pd.to_datetime('09:30').time(), 'cluster930'] = df_930['pct_diff_open_prev_close'].apply(assign_cluster)

# Fill NaN values in cluster930 column for non-9:30 AM rows
df['cluster930'] = df['cluster930'].fillna('N/A')

# Print the 9:30 AM data to verify
print(df[df['time'] == pd.to_datetime('09:30').time()][['date', 'time', 'pct_diff_open_prev_close', 'cluster930']])

              date      time  pct_diff_open_prev_close cluster930
15      2015-01-09  09:30:00                       NaN        N/A
390     2015-01-12  09:30:00                 -0.175719  -0.2--0.1
765     2015-01-13  09:30:00                  0.058848   -0.0-0.1
1140    2015-01-14  09:30:00                 -0.040343  -0.1--0.0
1515    2015-01-15  09:30:00                  1.554101    1.5-1.6
...            ...       ...                       ...        ...
888651  2024-08-22  09:30:00                  0.182593    0.1-0.2
889026  2024-08-23  09:30:00                  0.073556   -0.0-0.1
889401  2024-08-26  09:30:00                  0.345323    0.3-0.4
889776  2024-08-27  09:30:00                 -0.037183  -0.1--0.0
890151  2024-08-28  09:30:00                  0.062007   -0.0-0.1

[2374 rows x 4 columns]


In [29]:
clusters = [f"{i:.1f}-{i+0.1:.1f}" for i in np.arange(-10.0, 10.0, 0.1)]
cluster_df = pd.DataFrame({'Cluster': clusters})

In [30]:
df_930 = df[df['time'] == pd.to_datetime('09:30').time()]
cluster_counts = df_930['cluster930'].value_counts().reset_index()
cluster_counts.columns = ['Cluster', 'Count']

In [31]:
result_df = cluster_df.merge(cluster_counts, on='Cluster', how='left')
result_df['Count'] = result_df['Count'].fillna(0).astype(int)

In [32]:
from dash import Dash, dash_table, html
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H1("Cluster Counts for 9:30 AM", className="text-center mt-4 mb-4"),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in result_df.columns],
        data=result_df.to_dict('records'),
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={
            'textAlign': 'center',
            'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        page_action='none',
        fixed_rows={'headers': True},
    )
], fluid=True)

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>